# Function Calling 과 RAG 연동

In [1]:
from dotenv import load_dotenv
import os
import json

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

### Function Calling을 사용하지 않고 웹검색하기 : gpt-4o-mini 웹 검색 
https://platform.openai.com/docs/guides/tools-web-search?api-mode=responses&lang=python

(gpt-4o-mini 웹 검색 요금) <br>

- 웹 검색 도구 호출 (gpt-4o, gpt-4.1, gpt-4o-mini, gpt-4.1-mini 검색 프리뷰)  : US ＄25.00 / 1,000건 호출 ,  1건당 약 ＄0.025 (약 34원) <br>
  https://openai.com/ko-KR/api/pricing/
- 웹 검색 컨텐츠 토큰 : 검색으로 가져온 텍스트 토큰 비용 부과 안 됨.
- 응답 생성 토큰 : gpt-4o-mini 모델의 표준 토큰 요금 적용 (입력/출력 토큰 과금은 별도)
- 검색 프리뷰 모드는 자동으로 검색 결과를 사용하여 처리함.

In [12]:
response = client.responses.create(
    model="gpt-4o-mini",
    tools=[{"type":"web_search"}],
    input="기상청의 오늘의 서울 날씨는?"
    # input="2024 파리 올림픽 우승국은?"
    # input="오늘은 며칠이지?"
    # input="서울의 현재 시간은?"    
)

print(response.output_text)
# type : web_search, code_interpreter, file_search, function

서울의 오늘 날씨는 흐리고 비가 내리고 있습니다. 현재 기온은 약 18°C이며, 최고 기온은 22°C, 최저 기온은 18°C로 예상됩니다. 습도는 75%로 다소 높은 편입니다. 비는 오전 8시 45분경부터 시작될 것으로 보이며, 오후 12시까지 약 0.25인치(약 6.35mm)의 비가 내릴 것으로 예상됩니다. ([weather.com](https://weather.com/weather/today/l/18e81cdf57491c51a6fba3c57732b7b61bdf511fc2b613570316978b9f20687a?utm_source=openai))

바람은 동남동쪽에서 시속 약 8km로 불고 있으며, 자외선 지수는 1로 낮은 편입니다. 오늘의 일출은 오전 6시 35분, 일몰은 오후 6시 2분입니다. ([weather.com](https://weather.com/weather/today/l/18e81cdf57491c51a6fba3c57732b7b61bdf511fc2b613570316978b9f20687a?utm_source=openai))

오늘 하루 우산을 챙기시는 것이 좋겠습니다. 


### Function Calling을 사용한 날씨 알아오기
- wttr.in 무료 날씨 API 사용

In [29]:
import json
import requests
from datetime import datetime

def get_weather(location):
    print(f"🌤️ {location}의 날씨를 검색 중...")

    # wttr.in 무료 날씨 API 사용:https://wttr.in/seoul,https://wttr.in/guam
    url = f"https://wttr.in/{location}?format=j1"
    response = requests.get(url,timeout=5)

    if response.status_code == 200:
        data = response.json()
        current = data["current_condition"][0]

        return {
                "location": location,
                "temperature": f"{current['temp_C']}°C",
                "condition": current['weatherDesc'][0]['value'],
                "humidity": f"{current['humidity']}%"            
        }

    # 실패시 Mock 데이터 반환
    return {
        "location": location,
        "temperature": "15°C",
        "condition": "흐림",
        "humidity": "60%"
    }
        
def get_current_time():  # 현재 시간을 반환
    return datetime.now().strftime("%Y년 %m월 %d일 %H시 %M분")
    

# 사용 가능한 함수들
functions = {
    "get_weather": get_weather,
    "get_current_time": get_current_time
}   


In [34]:
# Function Call 스키마 정의
function_schemas = [
    {
        "name": "get_weather",
        "description": "특정 지역의 날씨 정보를 가져옵니다",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "날씨를 확인할 지역 (예: 서울, 부산, 뉴욕)"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "get_current_time",
        "description": "현재 시간을 가져옵니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    }
]

In [36]:
# # 파이썬의 일급 함수
# def add(a,b):
#     print('add function is called!!')
#     return a+b
    
# def subtract(a,b):
#     print('subtract function is called!!')
#     return a-b

# func = {'first':add,'second':subtract}

# func['first'](10,10)

# arg ={'a':10,'b':20}
# add(**arg)  # 키워드 인수 , add(a=10,b=20)

add function is called!!
add function is called!!


30

In [37]:
def chat_with_functions(user_message):
    """Function Call을 사용하여 대화하는 함수"""
    print(f"\n💬 사용자: {user_message}")
    
    # 1단계: GPT에게 어떤 함수를 호출할지 결정하게 하기
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 날씨와 시간 정보를 제공하는 도우미입니다."},
            {"role": "user", "content": user_message}
        ],
        tools=[{"type": "function", "function": schema} for schema in function_schemas],
        tool_choice="auto"
    )
    
    # 2단계: Function Call이 있는지 확인
    if response.choices[0].message.tool_calls:
        # 메시지 히스토리에 GPT 응답 추가
        messages = [
            {"role": "system", "content": "당신은 날씨와 시간 정보를 제공하는 도우미입니다."},
            {"role": "user", "content": user_message},
            response.choices[0].message
        ]
        
        # 3단계: 각 함수 호출 실행
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 함수 호출: {function_name}({function_args})")
            
            # 함수 실행
            if function_name in functions:
                result = functions[function_name](**function_args)
                
                # 함수 결과를 메시지에 추가
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(result, ensure_ascii=False)
                })
        
        # 4단계: 함수 결과를 바탕으로 최종 답변 생성
        final_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        
        return final_response.choices[0].message.content
    
    else:
        # Function Call이 없으면 바로 답변 반환
        return response.choices[0].message.content

In [38]:
def main():
    """메인 함수 - 간단한 데모"""
    print("=== OpenAI Function Call 간단한 예제 ===")
    print("날씨와 시간 정보를 제공합니다!")
    print("\n예시:")
    print("- '서울 날씨 어때?'")
    print("- '지금 몇 시야?'")
    print("- 'quit'으로 프로그램 종료\n")
    
    while True:
        user_input = input("질문: ").strip()
        
        if user_input.lower() in ['종료', 'quit', 'exit']:
            print("프로그램을 종료합니다.")
            break
        
        if user_input:
            answer = chat_with_functions(user_input)
            print(f"🤖 AI: {answer}\n")

if __name__ == "__main__":
    main()

=== OpenAI Function Call 간단한 예제 ===
날씨와 시간 정보를 제공합니다!

예시:
- '서울 날씨 어때?'
- '지금 몇 시야?'
- 'quit'으로 프로그램 종료



질문:  서울 날씨 어때?



💬 사용자: 서울 날씨 어때?
🔧 함수 호출: get_weather({'location': '서울'})
🌤️ 서울의 날씨를 검색 중...
🤖 AI: 현재 서울의 날씨는 다음과 같습니다:
- 온도: 16°C
- 날씨: 비
- 습도: 100%

외출 시 우산을 챙기시는 것이 좋습니다.



질문:  서울 날씨 어때?



💬 사용자: 서울 날씨 어때?
🔧 함수 호출: get_weather({'location': '서울'})
🌤️ 서울의 날씨를 검색 중...
🤖 AI: 현재 서울 날씨는 16°C이며, 비가 오고 있습니다. 습도는 100%입니다. 우산 잊지 마세요!



...


💬 사용자: 뉴욕의 현재 시간은?
🔧 함수 호출: get_current_time({})
🤖 AI: 현재 뉴욕의 시간은 2025년 10월 13일 20시 38분입니다.



질문:  안녕 나는 홍길동이야



💬 사용자: 안녕 나는 홍길동이야
🤖 AI: 안녕하세요, 홍길동님! 어떻게 도와드릴까요? 날씨나 현재 시간에 대한 정보가 필요하신가요?



질문:  quit


프로그램을 종료합니다.


### Function Calling과 RAG 연동 활용 

In [39]:
from dotenv import load_dotenv
import os

import pymupdf as fitz

import faiss
import numpy as np
import pickle

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 1)환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

In [42]:
# 2) 로컬 문서 (RAG 대상)
documents = [
    {"title": "제주 날씨", "chunk": "제주도는 해양성 기후로 연중 강수량이 많고 여름에 태풍 영향이 크다."},
    {"title": "서울 날씨", "chunk": "서울은 9월 말 선선하고 맑으며 낮 최고기온은 20~25도 사이이다."},
    {"title": "부산 날씨", "chunk": "부산은 남해안 특성상 겨울에도 비교적 따뜻하고 강수량이 많다."}
]

# 3) FAISS 인덱스 생성
embeddings = []
for doc in documents:
    emb = client.embeddings.create(
        model="text-embedding-3-small",
        input=doc["chunk"]
    )
    embeddings.append(emb.data[0].embedding)

emb_array = np.array(embeddings).astype("float32")
index = faiss.IndexFlatL2(len(emb_array[0]))
index.add(emb_array)

# 4) RAG 검색 함수
def search_similar_chunks(query, top_k=2):
    query_emb = client.embeddings.create(
        model="text-embedding-3-small",
        input=query
    ).data[0].embedding
    query_vec = np.array([query_emb]).astype("float32")
    distances, indices = index.search(query_vec, top_k)
    return [documents[i] for i in indices[0]]    

In [43]:
# 5) RAG 기반 답변 생성 함수
def generate_rag_answer(query, similar_chunks):
    """검색된 컨텍스트를 바탕으로 답변을 생성합니다."""
    context = "\n\n".join([chunk['chunk'] for chunk in similar_chunks])
    prompt = f"""다음 문서 내용을 참고하여 질문에 답변해주세요.

문서 내용:
{context}

질문: {query}

답변:"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=500
    )
    return response.choices[0].message.content

In [44]:
# 6) Function Calling 예시 (원하면 JSON 형태로 정리)
def generate_rag_answer_json(query, similar_chunks):
    """Function Calling을 이용해 JSON 형태로 응답"""
    context = "\n\n".join([chunk['chunk'] for chunk in similar_chunks])
    messages = [
        {"role": "system", "content": "당신은 질문에 대해 JSON 형식으로 답하는 어시스턴트입니다."},
        {"role": "user", "content": f"""
다음 문서를 참고하여 질문에 답해주세요.
문서:
{context}
질문: {query}
JSON 형식으로 답해주세요. (summary: 요약, source_titles: 참고 문서 제목 배열)
        """}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        functions=[
            {
                "name": "return_weather_info",
                "description": "질문에 맞는 날씨 정보를 JSON으로 반환",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "summary": {"type": "string"},
                        "source_titles": {"type": "array", "items": {"type": "string"}}
                    },
                    "required": ["summary", "source_titles"]
                }
            }
        ],
        function_call={"name": "return_weather_info"}  # ✅ 함수 호출 강제
    )
    return response.choices[0].message

# 7) 실행 예시
if __name__ == "__main__":
    query = "9월 말 서울 날씨 어때?"
    similar_chunks = search_similar_chunks(query)

    print("🔎 [일반 답변]")
    print(generate_rag_answer(query, similar_chunks))

    print("\n📝 [Function Calling JSON 응답]")
    print(generate_rag_answer_json(query, similar_chunks))

🔎 [일반 답변]
9월 말 서울은 선선하고 맑으며 낮 최고기온은 20~25도 사이입니다.

📝 [Function Calling JSON 응답]
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"summary":"서울은 9월 말 선선하고 맑으며 낮 최고기온은 20~25도 사이이다.","source_titles":["서울 날씨 정보"]}', name='return_weather_info'), tool_calls=None)


### DuckDuckGo로 웹 검색하기

In [45]:
# ! pip install duckduckgo-search
# ! pip install ddgs

In [47]:
from ddgs import DDGS

def search_duckduckgo(query, max_results=5):
    """DuckDuckGo로 간단한 웹 검색을 수행합니다."""
    try:
        with DDGS() as ddgs:
            print(f"🔍 '{query}' 검색 중...")
            print("=" * 50)
            
            results = ddgs.text(query, max_results=max_results)
            
            for i, result in enumerate(results, 1):
                print(f"\n[{i}] {result['title']}")
                print(f"🔗 {result['href']}")
                print(f"📝 {result['body'][:150]}...")
                print("-" * 30)
                
    except Exception as e:
        print(f"❌ 검색 오류: {e}")

# 실행 예시
if __name__ == "__main__":
    # 검색 예시들
    search_queries = [
        "ChatGPT 사용법",
        "서울 맛집 추천"
    ]
    
    for query in search_queries:
        search_duckduckgo(query, 3)
        print("\n" + "="*60 + "\n")
    
    # 직접 검색해보기
    user_query = input("검색어를 입력하세요: ")
    if user_query.strip():
        search_duckduckgo(user_query, 5)


🔍 'ChatGPT 사용법' 검색 중...

[1] Codestates ChatGPT 사용법 | 가입부터 확장 프로그램까지 200% 활용법
🔗 https://www.codestates.com/blog/content/chatgpt-사용법
📝 March 3, 2023 - 인공지능(AI) 챗봇 ChatGPT (챗지피티). ChatGPT를 아직 잘 모르는 분들, ChatGPT를 더 잘 쓰고 싶은 분들을 위해 ChatGPT 가입부터 확장 프로그램 사용법까지 ChatGPT 200% 활용법을 알려드릴게요....
------------------------------

[2] Namu Wiki ChatGPT - 나무위키
🔗 https://namu.wiki/w/ChatGPT
📝 5 days ago - 자세한 내용은 ChatGPT / 사용법 문서를 참고하십시오....
------------------------------

[3] ApiDog 개발자 모드에서 ChatGPT MCP 서버 지원 커넥터 사용법
🔗 https://apidog.com/kr/blog/chatgpt-mcp-support/
📝 September 12, 2025 - ChatGPT가 단순히 대화만 하는 것을 넘어, 당신의 도구와 데이터 소스에 손쉽게 연결하여 실시간 정보로 작업을 자동화할 수 있다면 어떨까요? 이것이 바로 AI 기반 자동화의 판도를 바꾸는 ChatGPT의 MCP 지원이 약속하는 ...
------------------------------


🔍 '서울 맛집 추천' 검색 중...

[1] YouTube 2년 동안 가봤던 서울 맛집 107곳 중 베스트 11 - YouTube
🔗 https://www.youtube.com/watch?v=qFnWlbC7kHA
📝 1. 깃대봉냉면 00:072. 동해루 01:393. 국보성 03:594. 천호동엄마손만두 05:595. 강원집(닭진미) 07:466. 어머니국시방 09:227. 대성원(대성반점) 10:378. 원두막 12:459. 졍동국시 16:3310. 삼경

검색어를 입력하세요:  랭체인 사용법


🔍 '랭체인 사용법' 검색 중...

[1] Wikipedia 랭체인 - 위키백과, 우리 모두의 백과사전
🔗 https://ko.wikipedia.org/wiki/랭체인
📝 June 17, 2025 - 2023년 4월 랭체인이 법인화되었고 새로운 스타트업은 벤치마크로부터 1,000만 달러의 초기 투자를 발표한 지 일주일 만에 벤처 회사 세쿼이아 캐피털로부터 최소 2억 달러의 가치로 2,000만 달러 이상의 자금을 조달했다....
------------------------------

[2] Samsung SDS 랭체인 LangChain 의 개념과 이해 | 인사이트리포트 | 삼성SDS
🔗 https://www.samsungsds.com/kr/insights/the-concept-of-langchain.html
📝 March 11, 2024 - LLM을 효과적으로 사용할 수 있는 프롬프트 작성에는 어느 정도 기술이 필요하지만, LLM 사용법은 대체로 간단합니다. 반면 언어 모델을 사용한 프로그래밍은 어려울 수 있습니다. 그럴 때는 랭체인을 사용하면 됩니다....
------------------------------

[3] Teddylee777 랭체인(langchain)의 OpenAI GPT 모델(ChatOpenAI) 사용법 (1) - 테디노트
🔗 https://teddylee777.github.io/langchain/langchain-tutorial-01/
📝 September 28, 2023 - 랭체인 (langchain)의 OpenAI GPT 모델(ChatOpenAI) 사용법을 함께 알아보겠습니다....
------------------------------

[4] ITWorld Korea “LLM 개발을 더 간편하게” 랭체인(LangChain)의 이해 - ITWorld Korea
🔗 https://www.itworld.co.kr/news/307189
📝 대규모 언어 모델(LLM)을 위한 효과적인 프롬프트를 작성하는 데는 기술이 필요하지만 LLM 사용법